In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math

from pandas import DataFrame, Series
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
def print_graph(MinMax, num): # num is cut range
    length = len(MinMax)
    plt.figure(figsize=(20, 5)) # graph size
    plt.xticks(np.arange(0, length+1, num))
    plt.plot(MinMax)
    plt.show()

In [3]:
def normalization(columns): # -1~1사이의 값으로 정규화
    MinMax = (columns - columns.min(axis=0)) / (columns.max(axis=0) - columns.min(axis=0))
    MinMax = (MinMax * 2) - 1
    MinMax
    return MinMax

In [4]:
def cut_count(MinMax, cnt): # 왼발, 오른발의 데이터를 잘라서 구분
    count = int((len(MinMax)) / cnt)
    step = []
    temp = 0
    for i in range(0, count):
        temp = i*cnt
        step.append(MinMax[temp:temp+cnt])
    return step

In [5]:
def ACF(step, cnt): #자기상관계수 구하는 함수 cnt = 한발에 들어가는 데이터의 수
    answer = 0
    acf = []
    for i in step:
        answer = 0
#         answer = np.linalg.norm(i)
        for j in i:
#             print(j)
            answer = answer + j*j
        answer = answer / cnt
#         print(answer)
        acf.append(answer)
    return acf

In [6]:
def compare_acf(acf): # 자기상관계수 값을 구해줌
    num = len(acf) / 2
    left = []
    right = []
    result = []
    for i in range(1, len(acf), +2):
        left.append(acf[i])
        right.append(acf[i-1])
    result = np.corrcoef(left, right)
    result = abs(result)
#     print (result)
    return result

In [7]:
def seperate_z(x) :
    if (x < 0.6):
        return False
    else : return True

In [8]:
def LPF(rawData):
    rawData['targetData'] = rawData['AccZ']

    ts = 0.01 # 100Hz
    tau = 0.1

    tmpResult = Series(np.zeros(len(rawData.targetData)))
    tmpResult[0] = rawData.targetData[0]

    for i in np.arange(1, len(rawData.targetData)):
        tmpResult[i] = (tau*tmpResult[i-1] + ts * rawData.targetData[i]) / (ts + tau)

    rawData['LPF'] = tmpResult

#     plt.plot(rawData.SampleRate, rawData.targetData, '#afafaf',  rawData.SampleRate, rawData.LPF_X, 'b')
#     plt.grid(True)
#     plt.show()
#     print(rawData)
    return rawData

In [9]:
def main(rawData, count):
    LPFData = LPF(rawData)
#     print(LPFData)
    MinMax = normalization(LPFData['LPF'])
#     print_graph(MinMax, 250)
    step = cut_count(MinMax, count)
#     print(step)
    acf = ACF(step, count)
#     print(acf)
    compare = compare_acf(acf)
#     print(compare)
    result = True
    if (compare[0][1] > 0.6):
        result=  True
    else : result = False
    if (result == True):
        print('\033[34m', result, '\033[0m', compare[0][1])
    else : print('\033[31m', result, '\033[0m', compare[0][1])

In [15]:
for i in range(0, 10):
    count = 0
    rawData = pd.read_csv(f'normal1/normalWalk{i}.csv')
    columns = rawData[['AccX', 'AccY', 'AccZ']]
    step_count = rawData['stepCount']
    for i in range(0, len(step_count)) :
        if (step_count[i] == 0) : 
            count = i
            break
#     print(round(len(step_count)/count))
    main(columns, round(len(step_count)/count))
# rawData = pd.read_csv('normal/normalWalk.csv')
#     columns = rawData[['AccX', 'AccY', 'AccZ']]
#     step_count = rawData['stepCount']
#     for i in range(0, len(step_count)) :
#         if (step_count[i] == 0) : 
#             count = i
#             break
# #     print(round(len(step_count)/count))
#     main(columns, round(len(step_count)/count))

 False  0.2934633794432357
 False  0.2240720624881119
 False  0.1328980101292385
 True  0.6674790123616903
 False  0.050415466926580915
 True  0.7158028749872866
 False  0.1197535797150948
 True  0.8625754772417213
 False  0.14074240238739003
 False  0.02291660952515961


In [87]:
for i in range(0, 10):
    count = 0
    rawData = pd.read_csv(f'abnormal/abnormalWalk{i}.csv')
    columns = rawData[['AccX', 'AccY', 'AccZ']]
    step_count = rawData['stepCount']
    for i in range(0, len(step_count)) :
        if (step_count[i] == 0) : 
            count = i
            break
    main(columns, round(len(step_count)/count))
    print(round(len(step_count)/count))
# rawData = pd.read_csv('abnormal/abnormalWalk0.csv')
# columns = rawData[['AccX', 'AccY', 'AccZ']]
# step_count = rawData['stepCount']
# for i in range(0, len(step_count)) :
#     if (step_count[i] == 0) : 
#         count = i
#         break
# main(columns, round(len(step_count)/count))
#     print(round(len(step_count)/count*2), count*2)

 False  0.25498365223088426
235
 False  0.18792281141838932
187
 False  0.5089077315694428
107
 True  0.7900517854362574
124
 False  0.16672283785250525
117
 True  1.0
332
 True  0.8126779694443639
126
 False  0.5239350175977812
144
 True  0.6083440797757602
125
 True  0.7733647282088848
202
